# Ninai Multimodal (Attachments) Demo

This notebook demonstrates a **multimodal memory case** using Ninai attachments:

- Create a long-term memory

- Upload 3 attachments (**DOCX + image + text**)

- Verify attachment indexing status (`indexed_at`)

- (Optional) Inspect stored vectors/payloads directly in **Qdrant**

## Prerequisites
- Backend running (default API base URL: `http://localhost:8000/api/v1`)
- Auth via API key (`NINAI_API_KEY`) or email/password
- Qdrant running (for indexing + optional inspection)

Note: Attachments are stored against **long-term** memories (Postgres). This demo uses `POST /memories` via the SDK’s `client.memories.create(...)`.

Note: The current `/memories/search` endpoint is a smoke test (it uses a placeholder embedding). This notebook still shows how to verify attachment indexing via `indexed_at` and by retrieving points from Qdrant.

### Optional: enable OCR for images
If your backend container does not have `tesseract`, enable the OCR sidecar so image attachments can be text-extracted:

```bash
docker compose -f docker-compose.yml -f docker-compose.ocr.yml up -d --build
```

Tip: If you use email/password, pressing Enter will re-prompt until the email looks valid.

In [10]:
# Cell 2: Install SDK + small helpers used to generate demo assets.
# - `ninai` is installed editable from this repo.
# - `python-docx` and `pillow` are only used client-side to create files to upload.
# - `qdrant-client` is only used in the optional Qdrant inspection cell.
%pip install -q -e ../sdk/python python-docx pillow qdrant-client
print("Installed dependencies")

Note: you may need to restart the kernel to use updated packages.
Installed dependencies


In [11]:
# Cell 3: Configure client + authenticate
import os
import re
from pathlib import Path
from typing import Any
from getpass import getpass

from ninai import NinaiClient

API_URL = os.getenv("NINAI_API_URL", "http://localhost:8000/api/v1")
API_KEY = os.getenv("NINAI_API_KEY")
ORG_ID = os.getenv("NINAI_ORGANIZATION_ID")

client = NinaiClient(base_url=API_URL, api_key=API_KEY, organization_id=ORG_ID)

if not client.is_authenticated:
    email_re = re.compile(r"^[^@\s]+@[^@\s]+\.[^@\s]+$")
    while True:
        email = input("Email: ").strip()
        if email_re.match(email):
            break
        print("Please enter a valid email address.")
    password = getpass("Password: ")
    user = client.login(email=email, password=password)
    print(f"Logged in as: {user.email}")
else:
    print("Client is authenticated (API key or existing token).")

print("API:", client.base_url)

Logged in as: admin@ninai.dev
API: http://localhost:8000/api/v1


In [12]:
# Cell 4: Create demo assets (DOCX + image + text)
import uuid

from docx import Document
from PIL import Image, ImageDraw

demo_dir = (Path.cwd() / "_demo_assets" / "multimodal_memory_demo").resolve()
demo_dir.mkdir(parents=True, exist_ok=True)

run_id = uuid.uuid4().hex[:8]
unique_token = f"NINAI_DEMO_TOKEN_{run_id}"
print("Demo dir:", demo_dir)
print("Unique token:", unique_token)

# 1) Text file
txt_path = demo_dir / f"customer_note_{run_id}.txt"
txt_content = "\n".join(
    [
        "Customer: ACME Corp",
        "Issue: refund requested for Order #777",
        f"UniqueToken: {unique_token}",
        "Resolution: offered replacement + expedited shipping",
    ]
)
txt_path.write_text(txt_content, encoding="utf-8")

# 2) DOCX file
docx_path = demo_dir / f"call_summary_{run_id}.docx"
doc = Document()
doc.add_heading("Support Call Summary", level=1)
doc.add_paragraph("Caller reported a billing discrepancy on invoice INV-2026-00042.")
doc.add_paragraph(f"Reference token for demo retrieval: {unique_token}")
doc.add_paragraph("Next step: verify refund eligibility and notify accounting.")
doc.save(str(docx_path))

# 3) Image file (OCR is optional server-side)
img_path = demo_dir / f"invoice_screenshot_{run_id}.png"
img = Image.new("RGB", (900, 240), color=(245, 245, 245))
draw = ImageDraw.Draw(img)
draw.text((24, 24), "Invoice: INV-2026-00042", fill=(20, 20, 20))
draw.text((24, 72), "Amount: $1,234.56", fill=(20, 20, 20))
draw.text((24, 120), f"Token: {unique_token}", fill=(20, 20, 20))
draw.text((24, 168), "(Enable OCR sidecar to extract text)", fill=(90, 90, 90))
img.save(str(img_path))

print("Created:")
print("-", txt_path.name)
print("-", docx_path.name)
print("-", img_path.name)

Demo dir: D:\Sansten\Projects\Ninai2\notebooks\_demo_assets\multimodal_memory_demo
Unique token: NINAI_DEMO_TOKEN_5dcb5419
Created:
- customer_note_5dcb5419.txt
- call_summary_5dcb5419.docx
- invoice_screenshot_5dcb5419.png


In [13]:
# Cell 5: Create a long-term memory to attach files to
memory = client.memories.create(
    title=f"Multimodal demo ({run_id})",
    content=
        "This memory demonstrates Ninai attachments. "
        "The searchable details are mostly inside the attachments.",
    tags=["demo", "multimodal", "attachments"],
    scope="personal",
    memory_type="long_term",
    classification="internal",
    metadata={"demo_run_id": run_id, "unique_token": unique_token},
 )

print("Created memory:", memory.id)

Created memory: f78b3e0e-7ad2-4eef-bd90-b96eec9053b1


In [14]:
# Cell 6: Upload attachments (DOCX + image + text)
paths = [docx_path, img_path, txt_path]
uploaded: list[dict[str, Any]] = []

for p in paths:
    resp = client.memories.upload_attachment(memory.id, str(p))
    uploaded.append(resp)
    print(f"Uploaded {p.name} -> attachment id: {resp.get('id')}")

attachment_ids = [u.get("id") for u in uploaded if u.get("id")]
print("Attachment IDs:", attachment_ids)

Uploaded call_summary_5dcb5419.docx -> attachment id: 6f6e460a-afbb-4bf7-b1ac-d56ebd1dc3db
Uploaded invoice_screenshot_5dcb5419.png -> attachment id: 7c59aa3c-f2fb-40ff-b387-3d1e71f0f287
Uploaded customer_note_5dcb5419.txt -> attachment id: d8f94935-1c22-474c-b0fc-1ecd7d9cc102
Attachment IDs: ['6f6e460a-afbb-4bf7-b1ac-d56ebd1dc3db', '7c59aa3c-f2fb-40ff-b387-3d1e71f0f287', 'd8f94935-1c22-474c-b0fc-1ecd7d9cc102']


In [15]:
# Cell 7: Verify indexing status
# The backend sets `indexed_at` when extracted text was embedded and written to Qdrant.
# For images, `indexed_at` may be null unless OCR is enabled.

def list_attachments() -> list[dict[str, Any]]:
    data = client.memories.list_attachments(memory.id)
    return list(data.get("items", []))

items = list_attachments()
print(f"Attachments: {len(items)}")
for a in items:
    print("-")
    print("  id:", a.get("id"))
    print("  file_name:", a.get("file_name"))
    print("  content_type:", a.get("content_type"))
    print("  size_bytes:", a.get("size_bytes"))
    print("  indexed_at:", a.get("indexed_at"))

Attachments: 3
-
  id: d8f94935-1c22-474c-b0fc-1ecd7d9cc102
  file_name: customer_note_5dcb5419.txt
  content_type: text/plain
  size_bytes: 153
  indexed_at: 2026-01-20T21:02:48.642721Z
-
  id: 7c59aa3c-f2fb-40ff-b387-3d1e71f0f287
  file_name: invoice_screenshot_5dcb5419.png
  content_type: image/png
  size_bytes: 7812
  indexed_at: 2026-01-20T21:02:48.610439Z
-
  id: 6f6e460a-afbb-4bf7-b1ac-d56ebd1dc3db
  file_name: call_summary_5dcb5419.docx
  content_type: application/vnd.openxmlformats-officedocument.wordprocessingml.document
  size_bytes: 36757
  indexed_at: 2026-01-20T21:02:48.442810Z


In [16]:
# Cell 8: (Optional) Call the /memories/search endpoint
# NOTE: In the current backend, /memories/search uses a placeholder embedding (all zeros),
# so results may NOT reflect the query string yet. Treat this as a smoke test only.
query = unique_token
results = client.memories.search(query, limit=5)

print(f"Search query (smoke test): {query}")
print(f"Results: {len(results.items)}")
for i, m in enumerate(results.items, start=1):
    score = getattr(m, "score", None)
    score_str = f"{score:.3f}" if isinstance(score, (int, float)) else "?"
    print(f"{i}. [{score_str}] {m.id} | {m.title}")
    print("   preview:", (m.content_preview or "")[:120])

Search query (smoke test): NINAI_DEMO_TOKEN_5dcb5419
Results: 0


In [17]:
# Cell 9 (Optional): Inspect attachment vectors in Qdrant
# Requires Qdrant exposed on localhost:6333 (docker-compose default)
import os
from qdrant_client import QdrantClient

QDRANT_HOST = os.getenv("QDRANT_HOST", "localhost")
QDRANT_PORT = int(os.getenv("QDRANT_PORT", "6333"))
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION_NAME", "memories")

qc = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
print("Qdrant:", QDRANT_HOST, QDRANT_PORT, "collection=", QDRANT_COLLECTION)

# Fetch points by attachment id; these points have payload.kind == 'attachment'
if not attachment_ids:
    raise RuntimeError("No attachment ids found")

points = qc.retrieve(collection_name=QDRANT_COLLECTION, ids=attachment_ids, with_payload=True, with_vectors=False)
for p in points:
    payload = p.payload or {}
    print("- point_id:", p.id)
    print("  kind:", payload.get("kind"))
    print("  memory_id:", payload.get("memory_id"))
    print("  file_name:", payload.get("file_name"))
    print("  content_type:", payload.get("content_type"))

Qdrant: localhost 6333 collection= memories
- point_id: 6f6e460a-afbb-4bf7-b1ac-d56ebd1dc3db
  kind: attachment
  memory_id: f78b3e0e-7ad2-4eef-bd90-b96eec9053b1
  file_name: call_summary_5dcb5419.docx
  content_type: application/vnd.openxmlformats-officedocument.wordprocessingml.document
- point_id: 7c59aa3c-f2fb-40ff-b387-3d1e71f0f287
  kind: attachment
  memory_id: f78b3e0e-7ad2-4eef-bd90-b96eec9053b1
  file_name: invoice_screenshot_5dcb5419.png
  content_type: image/png
- point_id: d8f94935-1c22-474c-b0fc-1ecd7d9cc102
  kind: attachment
  memory_id: f78b3e0e-7ad2-4eef-bd90-b96eec9053b1
  file_name: customer_note_5dcb5419.txt
  content_type: text/plain


In [18]:
# Cell 10 (Optional): Cleanup
# Set DO_CLEANUP=1 in your environment to delete the demo data.
DO_CLEANUP = os.getenv("DO_CLEANUP", "0") == "1"

if DO_CLEANUP:
    # Delete attachments (best effort)
    data = client.memories.list_attachments(memory.id)
    for a in data.get("items", []):
        client.memories.delete_attachment(memory.id, a["id"])
        print("Deleted attachment:", a["id"])

    client.memories.delete(memory.id)
    print("Deleted memory:", memory.id)
else:
    print("Skipping cleanup. To delete demo data: set DO_CLEANUP=1 and re-run this cell.")

Skipping cleanup. To delete demo data: set DO_CLEANUP=1 and re-run this cell.
